# PyTorch Global, Unstructured, Absolute Magnitude & Iterative Pruning:

Using _ResNet-18_ CNN trained from scratch on CIFAR-10 dataset.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.nn.utils.prune as prune

import sklearn
from sklearn.metrics import classification_report

# import matplotlib.pyplot as plt
import numpy as np

import copy, pickle, os

In [2]:
print(f"PyTorch Version: {torch.__version__}")
print(f"Torchvision Version: {torchvision.__version__}")

PyTorch Version: 1.8.1+cu101
Torchvision Version: 0.9.1+cu101


In [3]:
# Device configuration-
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"currently available device: {device}")

currently available device: cuda


In [ ]:
%env CUDA_DEVICE_ORDER = PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES = 2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [4]:
# Hyper-parameters-
num_epochs = 100
batch_size = 128
learning_rate = 0.1

In [5]:
# Define transformations for training and test sets-
transform_train = transforms.Compose(
    [
      transforms.RandomCrop(32, padding = 4),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
     ]
     )

transform_test = transforms.Compose(
    [
      transforms.ToTensor(),
      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
     ]
     )

In [ ]:
# os.chdir("/home/arjun/Documents/Programs/Python_Codes/PyTorch_Resources/Good_Codes/")

In [6]:
# Load dataset-
train_dataset = torchvision.datasets.CIFAR10(
        root = './data', train = True,
        download = True, transform = transform_train
        )

test_dataset = torchvision.datasets.CIFAR10(
        root = './data', train = False,
        download = True, transform = transform_test
        )


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
print(f"len(train_dataset) = {len(train_dataset)} & len(test_dataset) = {len(test_dataset)}")

len(train_dataset) = 50000 & len(test_dataset) = 10000


In [8]:
# Create training and testing loaders-
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size = batch_size,
        shuffle = True
        )

test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size = batch_size,
        shuffle = False
        )

In [9]:
print(f"len(train_loader) = {len(train_loader)} & len(test_loader) = {len(test_loader)}")

len(train_loader) = 391 & len(test_loader) = 79


In [10]:
# Sanity check-
len(train_dataset) / batch_size, len(test_dataset) / batch_size

(390.625, 78.125)

In [ ]:
# Get some random training images-
# some_img = iter(train_loader)
# images, labels = some_img.next()
images, labels = next(iter(train_loader))

# You get 32 images due to our specified batch size-
print(f"images.shape: {images.shape} & labels.shape: {labels.shape}")

images.shape: torch.Size([128, 3, 32, 32]) & labels.shape: torch.Size([128])


### ResNet model definition:

In [11]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


In [12]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


In [13]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


In [14]:
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [ ]:
def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])

In [ ]:
def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])

In [ ]:
def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])

In [ ]:
def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])

In [ ]:
import gc

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
foo = torch.tensor([1,2,3])
foo = foo.to('cuda')

In [15]:
# Initialize ResNet-18 CNN model-
trained_model = ResNet18()

In [16]:
import os
os.getcwd()

'/content'

In [17]:

# Move file from Google Colab to drive-
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [18]:
# !cp -rv ResNet18_best_trained_loss.pth drive/MyDrive
!cp -rv /content/drive/MyDrive/ResNet18_best_trained_loss.pth /content

'/content/drive/MyDrive/ResNet18_best_trained_loss.pth' -> '/content/ResNet18_best_trained_loss.pth'


In [ ]:
os.getcwd()

'/content'

In [19]:
os.listdir()

['.config', 'data', 'ResNet18_best_trained_loss.pth', 'drive', 'sample_data']

In [ ]:
# Path to file-
# PATH = "/home/arjun/Deep_Learning_Resources/Computer_Vision_Resources/ResNet_resources/ResNet_Codes/Good_Codes/ResNet18-CosineAnnealing_LR_scheduler/ResNet18_best_trained_loss.pth"
# PATH = "Weights/"
PATH = "/content/"

In [20]:
# Load pre-trained weights-
# trained_model.load_state_dict(torch.load(PATH + 'ResNet18_best_trained_loss.pth', map_location = device))
trained_model.load_state_dict(torch.load('ResNet18_best_trained_loss.pth', map_location = device))

<All keys matched successfully>

In [21]:
device

device(type='cuda')

In [ ]:
# Place model on GPU (if available)-
trained_model.to(device)

In [24]:
print(trained_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

### Custom absolute values weight pruning:

All weights less than a computed _threshold_ should be pruned away (or, set to zero) by subclassing ```prune.BasePruningMethod```. The pruning is: _unstructured, global_ and _iterative_.


__Refer__:

- [Lei Mao's Pytorch Pruning Tutorial](https://leimao.github.io/blog/PyTorch-Pruning/)

- [GitHub repo](https://github.com/leimao/PyTorch-Pruning-Example)

In [25]:
class ThresholdPruning(prune.BasePruningMethod):
    """
    Custom class to prune weights less than 'threshold'
    """
    PRUNING_TYPE = "unstructured"
    '''
    'PRUNING_TYPE' can be one of: global, structured or unstructured.
    
    'global' acts across whole module (e.g. remove 20% of weight with smallest value),
    'structured' acts on whole channels/modules.
    
    We need unstructured as we would like to modify each connection in specific parameter tensor (say 'weight' or 'bias')
    '''

    def __init__(self, threshold):
        self.threshold = threshold

    def compute_mask(self, tensor, default_mask):
        '''
        mask to be used to prune specific tensor. In our case all parameters below 'threshold' should be 0. This is done
        with absolute value as it makes more sense.
        
        'default_mask' is not needed here, but is left as named parameter as that's what API requires atm.
        '''
        return torch.abs(tensor) > self.threshold



In [26]:
def compute_global_threshold(model, p = 15):
    '''
    Function to compute global threshold for smallest magnitude pth percentile of weights
    '''
    # Python3 dict to hold weights of trained model-
    model_wts = {}
    
    for layer, params in model.named_parameters():
        model_wts[layer] = params.detach().cpu().numpy()
    
    # Python3 list to hold absolute magnitude flattened weights-
    flattened_wts = []
    
    '''
    for layer, params in model.named_parameters():
        # print(f"layer: {layer} has {params.shape}")
        flattened_wts.append(np.abs(params.detach().numpy().flatten()))
    '''
    
    for layer in model_wts.keys():
        flattened_wts.append(np.abs(model_wts[layer].flatten()))
    
    # Compute threshold using all weights from model-
    threshold = np.percentile(np.concatenate(flattened_wts), p)
    # print(f"threshold for {p}th percentile = {threshold:.4f}")
    
    return threshold


In [27]:
threshold = compute_global_threshold(trained_model, p = 20)
print(f"threshold for 20th percentile = {threshold}")

threshold for 20th percentile = 2.931090818947269e-07


In [29]:
threshold = compute_global_threshold(trained_model, p = 90)
print(f"threshold for 20th percentile = {threshold}")

threshold for 20th percentile = 1.5760478891024846e-05


In [30]:
def evaluate_model(model, test_loader, device, criterion = None):

    model.eval()
    model.to(device)

    running_loss = 0
    running_corrects = 0

    for inputs, labels in test_loader:

        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        if criterion is not None:
            loss = criterion(outputs, labels).item()
        else:
            loss = 0

        # statistics
        running_loss += loss * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    eval_loss = running_loss / len(test_loader.dataset)
    eval_accuracy = running_corrects / len(test_loader.dataset)

    return eval_loss, eval_accuracy


In [31]:
def create_classification_report(model, device, test_loader):

    model.eval()
    model.to(device)

    y_pred = []
    y_true = []

    with torch.no_grad():
        for data in test_loader:
            y_true += data[1].numpy().tolist()
            images, _ = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            y_pred += predicted.cpu().numpy().tolist()

    classification_report = sklearn.metrics.classification_report(
        y_true = y_true, y_pred = y_pred)

    return classification_report


In [32]:
def remove_parameters(model):

    for module_name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            try:
                prune.remove(module, "weight")
            except:
                pass
            try:
                prune.remove(module, "bias")
            except:
                pass
        elif isinstance(module, torch.nn.Linear):
            try:
                prune.remove(module, "weight")
            except:
                pass
            try:
                prune.remove(module, "bias")
            except:
                pass

    return model


In [33]:
def compute_final_pruning_rate(pruning_rate, num_iterations):
    '''
    A function to compute the final pruning rate for iterative pruning.
        Note that this cannot be applied for global pruning rate if the pruning rate is heterogeneous among different layers.
    Args:
        pruning_rate (float): Pruning rate.
        num_iterations (int): Number of iterations.
    Returns:
        float: Final pruning rate.
    '''

    final_pruning_rate = 1 - (1 - pruning_rate) ** num_iterations

    return final_pruning_rate


In [34]:
def measure_module_sparsity(module, weight=True, bias=False, use_mask=False):

    num_zeros = 0
    num_elements = 0

    if use_mask == True:
        for buffer_name, buffer in module.named_buffers():
            if "weight_mask" in buffer_name and weight == True:
                num_zeros += torch.sum(buffer == 0).item()
                num_elements += buffer.nelement()
            if "bias_mask" in buffer_name and bias == True:
                num_zeros += torch.sum(buffer == 0).item()
                num_elements += buffer.nelement()
    else:
        for param_name, param in module.named_parameters():
            if "weight" in param_name and weight == True:
                num_zeros += torch.sum(param == 0).item()
                num_elements += param.nelement()
            if "bias" in param_name and bias == True:
                num_zeros += torch.sum(param == 0).item()
                num_elements += param.nelement()

    sparsity = num_zeros / num_elements

    return num_zeros, num_elements, sparsity


In [35]:
def measure_global_sparsity(
    model, weight = True,
    bias = False, conv2d_use_mask = False,
    linear_use_mask = False):

    num_zeros = 0
    num_elements = 0

    for module_name, module in model.named_modules():

        if isinstance(module, torch.nn.Conv2d):

            module_num_zeros, module_num_elements, _ = measure_module_sparsity(
                module, weight=weight, bias=bias, use_mask=conv2d_use_mask)
            num_zeros += module_num_zeros
            num_elements += module_num_elements

        elif isinstance(module, torch.nn.Linear):

            module_num_zeros, module_num_elements, _ = measure_module_sparsity(
                module, weight=weight, bias=bias, use_mask=linear_use_mask)
            num_zeros += module_num_zeros
            num_elements += module_num_elements

    sparsity = num_zeros / num_elements

    return num_zeros, num_elements, sparsity


### torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones, gamma=0.1, last_epoch=-1, verbose=False)
[refer](https://pytorch.org/docs/stable/optim.html)

Decays the learning rate of each parameter group by 'gamma' once the number of epoch reaches one of the milestones. Notice that such decay can happen simultaneously with other changes to the learning rate from outside this scheduler. When ```last_epoch = -1```, sets initial lr as lr.

Parameters:

- optimizer (Optimizer) – Wrapped optimizer.

- milestones (list) – List of epoch indices. Must be increasing.

- gamma (float) – Multiplicative factor of learning rate decay. Default: 0.1.

- last_epoch (int) – The index of last epoch. Default: -1.

- verbose (bool) – If True, prints a message to stdout for each update. Default: False.

Example:
```
# Assuming optimizer uses lr = 0.05 for all groups
>>> # lr = 0.05     if epoch < 30
>>> # lr = 0.005    if 30 <= epoch < 80
>>> # lr = 0.0005   if epoch >= 80
>>> scheduler = MultiStepLR(optimizer, milestones=[30,80], gamma=0.1)
>>> for epoch in range(100):
>>>     train(...)
>>>     validate(...)
>>>     scheduler.step()
```

In [36]:
def fine_tune_train_model(model, train_loader, test_loader, device, l1_regularization_strength = 0,
                l2_regularization_strength = 1e-4, learning_rate = 1e-1, num_epochs = 20):

    # The training configurations were not carefully selected.

    criterion = nn.CrossEntropyLoss()

    model.to(device)

    # It seems that SGD optimizer is better than Adam optimizer for ResNet18 training on CIFAR10-
    optimizer = torch.optim.SGD(
        model.parameters(), lr = learning_rate,
        momentum = 0.9, weight_decay = l2_regularization_strength
    )
    # optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
    
    # Define learning rate scheduler-
    scheduler = torch.optim.lr_scheduler.MultiStepLR(
        # optimizer, milestones = [100, 150],
        optimizer, milestones = [5, 10],
        gamma = 0.1, last_epoch = -1)
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=500)
    

    # Evaluation-
    model.eval()
    eval_loss, eval_accuracy = evaluate_model(
        model = model, test_loader = test_loader,
        device = device, criterion = criterion)
    
    print(f"Pre fine-tuning: val_loss = {eval_loss:.3f} & val_accuracy = {eval_accuracy * 100:.3f}%")
    # print("Epoch: {:03d} Eval Loss: {:.3f} Eval Acc: {:.3f}".format(0, eval_loss, eval_accuracy))

    
    for epoch in range(num_epochs):

        # Training
        model.train()

        running_loss = 0
        running_corrects = 0

        for inputs, labels in train_loader:

            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            l1_reg = torch.tensor(0.).to(device)
            for module in model.modules():
                mask = None
                weight = None
                for name, buffer in module.named_buffers():
                    if name == "weight_mask":
                        mask = buffer
                for name, param in module.named_parameters():
                    if name == "weight_orig":
                        weight = param
                # We usually only want to introduce sparsity to weights and prune weights.
                # Do the same for bias if necessary.
                if mask is not None and weight is not None:
                    l1_reg += torch.norm(mask * weight, 1)

            loss += l1_regularization_strength * l1_reg

            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = running_corrects / len(train_loader.dataset)

        # Evaluation
        model.eval()
        eval_loss, eval_accuracy = evaluate_model(
            model = model, test_loader = test_loader,
            device = device, criterion = criterion)

        # Set learning rate scheduler
        scheduler.step()

        '''
        print(
            "Epoch: {:03d} Train Loss: {:.3f} Train Acc: {:.3f} Eval Loss: {:.3f} Eval Acc: {:.3f}"
            .format(epoch + 1, train_loss, train_accuracy, eval_loss,
                    eval_accuracy))
        '''
        print(f"epoch = {epoch + 1} loss = {train_loss:.3f}, accuracy = {train_accuracy * 100:.3f}%, val_loss = {eval_loss:.3f}, val_accuracy = {eval_accuracy * 100:.3f}% & LR: {optimizer.param_groups[0]['lr']:.4f}")

    return model


In [ ]:
# Sanity check-
# fine_tuned_model = fine_tune_train_model(trained_model, train_loader, test_loader, device)

### Available Pruning Methods:

The following child classes inherit from the ```BasePruningMethod```:

- ```torch.nn.utils.prune.Identity```: utility pruning method that does not prune any units but generates the pruning parametrization with a mask of ones;

- ```torch.nn.utils.prune.RandomUnstructured```: prune (currently unpruned) entries in a tensor at random;

- ```torch.nn.utils.prune.L1Unstructured```: prune (currently unpruned) entries in a tensor __by zeroing out the ones with the lowest absolute magnitude__;

- ```torch.nn.utils.prune.RandomStructured```: prune entire (currently unpruned)rows or columns in a tensor random;

- ```torch.nn.utils.prune.LnStructured```:  prune entire (currently unpruned) rows or columns in a tensor based on their $L_n$-norm (supported values of _n_ correspond to sup-ported values for argument _p_ in ```torch.norm()```);

- ```torch.nn.utils.prune.CustomFromMask```: prune a tensor using a user-provided mask.


Refer to "STREAMLINING TENSOR AND NETWORK PRUNING IN PYTORCH" by Michela Paganini et al. research paper for more details.

In [37]:
def save_model(model, model_dir, model_filename):
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
        
    model_filepath = os.path.join(model_dir, model_filename)
    torch.save(model.state_dict(), model_filepath)
    
    return None


In [38]:
def load_model(model, model_filepath, device):

    model.load_state_dict(torch.load(model_filepath, map_location=device))

    return model

In [39]:
pruning_rates = []

In [40]:
# Compute final pruning rate using 20% global unstructured pruning with 21 such pruning rounds-
final_pruning_rate = compute_final_pruning_rate(pruning_rate = 0.2, num_iterations = 21)

In [41]:
for i in range(1, 22):
    final_pruning_rate = compute_final_pruning_rate(pruning_rate = 0.2, num_iterations = i)
    pruning_rates.append(final_pruning_rate * 100)
    # print(f"ResNet-18 pruning round = {i} & pruning rate = {final_pruning_rate * 100:.3f}%")

In [42]:
pruning_rates

[19.999999999999996,
 35.999999999999986,
 48.79999999999999,
 59.03999999999999,
 67.23199999999999,
 73.78559999999999,
 79.02847999999999,
 83.22278399999999,
 86.57822719999999,
 89.26258175999999,
 91.410065408,
 93.1280523264,
 94.50244186112,
 95.601953488896,
 96.48156279111679,
 97.18525023289344,
 97.74820018631475,
 98.1985601490518,
 98.55884811924143,
 98.84707849539315,
 99.07766279631453]

In [43]:
def iterative_pruning_finetuning(
    model, train_loader, test_loader, device,
    learning_rate, l1_regularization_strength,
    l2_regularization_strength, learning_rate_decay = 0.1,
    conv2d_prune_amount = 0.2, linear_prune_amount = 0.1,
    num_iterations = 10, num_epochs_per_iteration = 10,
    model_filename_prefix = "pruned_model", model_dir = "saved_models",
    grouped_pruning = False):
    
    '''
    num_iterations - number of pruning iterations/rounds
    num_epochs_per_iteration - number of fine-tuning rounds
    '''
    
    best_val_acc = 0

    for i in range(1, num_iterations + 1):

        print("\nPruning and Finetuning {}/{}".format(i, num_iterations))

        print("Pruning...")


        # NOTE: For global pruning, linear/dense layer can also be pruned!
        if grouped_pruning == True:
            # grouped_pruning -> Global pruning
            parameters_to_prune = []
            for module_name, module in model.named_modules():
                if isinstance(module, torch.nn.Conv2d):
                    parameters_to_prune.append((module, "weight"))
                elif isinstance(module, torch.nn.Linear):
                    parameters_to_prune.append((module, "weight"))
            
            '''
            # L1Unstructured - prune (currently unpruned) entries in a tensor by zeroing
            # out the ones with the lowest absolute magnitude-
            prune.global_unstructured(
                parameters_to_prune,
                pruning_method = prune.L1Unstructured,
                amount = conv2d_prune_amount,
            )
            '''
            
            # Use custom absolute magnitude weight based 'global', 'unstructured' and 'iterative' pruning:
            
            # Compute threshold-
            computed_threshold = compute_global_threshold(model, p = pruning_rates[i - 1])
            # print(f"threshold for 20th percentile = {threshold}")

            prune.global_unstructured(
                parameters_to_prune,
                pruning_method = ThresholdPruning,
                threshold = computed_threshold
            )
        
        # layer-wise pruning-
        else:
            for module_name, module in model.named_modules():
                if isinstance(module, torch.nn.Conv2d):
                    prune.l1_unstructured(
                        module, name = "weight",
                        amount = conv2d_prune_amount)
                elif isinstance(module, torch.nn.Linear):
                    prune.l1_unstructured(
                        module, name = "weight",
                        amount = linear_prune_amount)

        # Compute validation accuracy just after pruning-
        _, eval_accuracy = evaluate_model(
            model = model, test_loader = test_loader,
            device = device, criterion = None)

        '''
        classification_report = create_classification_report(
            model=model, test_loader=test_loader, device=device)
        '''

        # Compute global sparsity-
        num_zeros, num_elements, sparsity = measure_global_sparsity(
            model, weight = True,
            bias = False, conv2d_use_mask = True,
            linear_use_mask = False)
        
        print(f"Global sparsity = {sparsity * 100:.3f}% & val_accuracy = {eval_accuracy * 100:.3f}%")
        # print(model.conv1._forward_pre_hooks)

        print("\nFine-tuning...")

        # train_model(
        fine_tuned_model = fine_tune_train_model(
            model = model, train_loader = train_loader,
            test_loader = test_loader, device = device,
            l1_regularization_strength = l1_regularization_strength,
            l2_regularization_strength = l2_regularization_strength,
            # i -> current pruning round-
            # learning_rate = learning_rate * (learning_rate_decay ** i),
            learning_rate = 1e-1,
            num_epochs = num_epochs_per_iteration)

        _, eval_accuracy = evaluate_model(
            model=model, test_loader = test_loader,
            device = device, criterion = None)

        '''
        classification_report = create_classification_report(
            model=model, test_loader=test_loader, device=device)
        '''

        num_zeros, num_elements, sparsity = measure_global_sparsity(
            # model,
            model, weight = True,
            bias = False, conv2d_use_mask = True,
            linear_use_mask = False)

        print(f"Post fine-tuning: Global sparsity = {sparsity * 100:.3f}% & val_accuracy = {eval_accuracy * 100:.3f}%")

        '''
        model_filename = "{}_{}.pt".format(model_filename_prefix, i + 1)
        model_filepath = os.path.join(model_dir, model_filename)
        save_model(model=model,
                   model_dir=model_dir,
                   model_filename=model_filename)
        model = load_model(model=model,
                           model_filepath=model_filepath,
                           device=device)
        '''
        
        '''
        # Save best performing model yet-
        if eval_accuracy > best_val_acc:
            best_val_acc = eval_accuracy
            print(f"\nSaving ResNet-18 sparsity = {sparsity * 100:.3f}%")
            torch.save(model.state_dict(), f'ResNet18_trained_sparsity-{sparsity * 100:.3f}.pth')
        '''
        
        
    return model


In [44]:
# num_classes = 10
# random_seed = 1
l1_regularization_strength = 0
l2_regularization_strength = 1e-4
learning_rate = 1e-3
learning_rate_decay = 1

In [45]:
_, eval_accuracy = evaluate_model(
    model = trained_model, test_loader=test_loader,
    device = device, criterion = None)

In [ ]:
'''
classification_report = create_classification_report(
    model = trained_model, test_loader = test_loader,
    device = device)
'''

In [46]:
num_zeros, num_elements, sparsity = measure_global_sparsity(trained_model)
print(f"Global sparsity = {sparsity:.3f}% & val_accuracy = {eval_accuracy * 100:.3f}%")

# print("Test Accuracy: {:.3f}".format(eval_accuracy))
# print("Classification Report:")
# print(classification_report)
# print("Global Sparsity:")
# print("{:.2f}".format(sparsity))

Global sparsity = 0.000% & val_accuracy = 88.990%


In [47]:
print(f"ResNet-18 has {num_elements} trainable parameters in it")

ResNet-18 has 11164352 trainable parameters in it


In [48]:
model_dir = "saved_models"
model_filename = "resnet18_cifar10.pth"
model_filename_prefix = "pruned_model"
pruned_model_filename = "resnet18_pruned_cifar10.pth"
model_filepath = os.path.join(model_dir, model_filename)
pruned_model_filepath = os.path.join(model_dir, pruned_model_filename)


In [49]:
print(f"ResNet-18 final pruning rate = {final_pruning_rate * 100:.3f}%")

ResNet-18 final pruning rate = 99.078%


In [50]:
import copy

In [51]:
print("Iterative Pruning + Fine-Tuning...")

Iterative Pruning + Fine-Tuning...


In [52]:
# Create a deep copy of the pre-trained model-
pruned_model = copy.deepcopy(trained_model)


# Prune and fine-tune trained model-
'''
num_iterations - number of pruning iterations/rounds
num_epochs_per_iteration - number of fine-tuning rounds

NOTE:
ResNt-18 when pruned @ 20% per global pruning round for 21 such rounds leads to 99.078% global sparsity.
'''
pruned_model = iterative_pruning_finetuning(
        model = pruned_model, train_loader = train_loader,
        test_loader = test_loader, device = device,
        learning_rate = learning_rate, learning_rate_decay = learning_rate_decay,
        l1_regularization_strength = l1_regularization_strength, l2_regularization_strength = l2_regularization_strength,
        conv2d_prune_amount = 0.2, linear_prune_amount = 0.1,
        num_iterations = 21, num_epochs_per_iteration = 12,
        model_filename_prefix = model_filename_prefix, model_dir = model_dir,
        grouped_pruning = True)


# Apply pruned mask to the parameters/weights and remove the masks-
remove_parameters(model = pruned_model)

_, eval_accuracy = evaluate_model(
    model = pruned_model, test_loader = test_loader,
    device = device, criterion = None
)

'''
classification_report = create_classification_report(
    model = pruned_model, test_loader = test_loader,
    device = device)
'''

num_zeros, num_elements, sparsity = measure_global_sparsity(pruned_model)


print(f"Global sparsity = {sparsity:.3f} & val_accuracy = {eval_accuracy:.3f}")
# print("Classification Report:")
# print(classification_report)
# NOTE: classification report is avoided as it's too verbose!



Pruning and Finetuning 2/21
Pruning...
Global sparsity = 20.016% & val_accuracy = 88.990%

Fine-tuning...
Pre fine-tuning: val_loss = 0.355 & val_accuracy = 88.990%
epoch = 1 loss = 0.577, accuracy = 80.102%, val_loss = 0.619, val_accuracy = 80.000% & LR: 0.1000
epoch = 2 loss = 0.487, accuracy = 83.072%, val_loss = 0.527, val_accuracy = 82.470% & LR: 0.1000
epoch = 3 loss = 0.449, accuracy = 84.490%, val_loss = 0.547, val_accuracy = 82.460% & LR: 0.1000
epoch = 4 loss = 0.429, accuracy = 85.132%, val_loss = 0.511, val_accuracy = 82.690% & LR: 0.1000
epoch = 5 loss = 0.409, accuracy = 85.854%, val_loss = 0.469, val_accuracy = 84.380% & LR: 0.0100
epoch = 6 loss = 0.298, accuracy = 89.846%, val_loss = 0.359, val_accuracy = 88.040% & LR: 0.0100
epoch = 7 loss = 0.265, accuracy = 90.762%, val_loss = 0.350, val_accuracy = 88.310% & LR: 0.0100
epoch = 8 loss = 0.253, accuracy = 91.182%, val_loss = 0.349, val_accuracy = 88.740% & LR: 0.0100
epoch = 9 loss = 0.246, accuracy = 91.512%, val_lo

In [53]:
# Remove pruning parameters-
final_model = remove_parameters(pruned_model)

In [54]:
# Compute final model's val_accuracy and global sparsity-
_, eval_accuracy = evaluate_model(
    model = final_model, test_loader = test_loader,
    device = device, criterion = None)

num_zeros, num_elements, sparsity = measure_global_sparsity(pruned_model)
print(f"Global sparsity = {sparsity * 100:.3f}%"
f" & val_accuracy = {eval_accuracy * 100:.3f}%")


Global sparsity = 99.099% & val_accuracy = 91.260%


In [55]:
# Save final trained and pruned model for later use-
torch.save(final_model.state_dict(), f"ResNet18_trained_sparsity-{sparsity * 100:.3f}.pth")

### Sanity check:

In [56]:
# Initialize and load trained and pruned model-
trained_pruned_model = ResNet18()
trained_pruned_model.load_state_dict(torch.load("/content/ResNet18_trained_sparsity-99.099.pth", map_location = device))
# trained_pruned_model.load_state_dict(final_model)

# Move model to GPU (if available)-
trained_pruned_model.to(device)

# Define cost function and optimizer-
criterion = nn.CrossEntropyLoss()

# It seems that SGD optimizer is better than Adam optimizer for ResNet18 training on CIFAR10-
optimizer = torch.optim.SGD(
        trained_pruned_model.parameters(), lr = learning_rate,
        momentum = 0.9, weight_decay = l2_regularization_strength
)


In [57]:
# Compute final model's val_accuracy and global sparsity-
eval_loss, eval_accuracy = evaluate_model(
    model = trained_pruned_model, test_loader=test_loader,
    device = device, criterion = None)

num_zeros, num_elements, sparsity = measure_global_sparsity(trained_pruned_model)
print(f"Global sparsity = {sparsity * 100:.3f}%, val_loss = {eval_loss:.3f}"
f" & val_accuracy = {eval_accuracy * 100:.3f}%")


Global sparsity = 99.099%, val_loss = 0.000 & val_accuracy = 91.260%
